# Problem Description
Click [here](https://code.aliyun.com/mindopt001/mindopt-open-examples/blob/master/doc/example_1.md?spm=5176.21102998.J_6338734050.1.637f5e8dfU1WxM&file=example_1.md).

# About
- Enviroment: Anaconda, python 3.8.10 64-bit, Windows 10.
- Using main algorithm: Taboo search.

## Solution Form
The best form for solutions is a numpy matrix. Considering the possible modification in the future, I wrapped it with `class solution`.

In [ ]:
import numpy as np

class solution(np.ndarray):
    pass